In [35]:
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
from tqdm import tqdm


def find_coco_json(folder):
    for f in os.listdir(folder):
        if f.endswith(".json"):
            return os.path.join(folder, f)
    raise FileNotFoundError(f"No COCO JSON found in {folder}")


def find_images_dir(folder):
    for c in ["images", "_images"]:
        path = os.path.join(folder, c)
        if os.path.isdir(path):
            return path
    raise FileNotFoundError(f"No images folder found in {folder}")


def coco_to_masks(split_dir):
    images_dir = find_images_dir(split_dir)
    masks_dir = os.path.join(split_dir, "masks")
    os.makedirs(masks_dir, exist_ok=True)

    coco_json = find_coco_json(split_dir)

    print(f"📄 Annotation: {coco_json}")
    print(f"🖼 Images: {images_dir}")

    coco = COCO(coco_json)

    for img_id in tqdm(coco.imgs, desc=f"Processing {split_dir}"):
        img_info = coco.imgs[img_id]
        h, w = img_info["height"], img_info["width"]

        mask = np.zeros((h, w), dtype=np.uint8)

        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        for ann in anns:
            ann_mask = coco.annToMask(ann)
            mask = np.maximum(mask, ann_mask)

        mask = (mask * 255).astype(np.uint8)

        img_name = img_info["file_name"]
        mask_name = os.path.splitext(os.path.basename(img_name))[0] + ".png"

        cv2.imwrite(os.path.join(masks_dir, mask_name), mask)

    print(f"✅ Masks saved in {masks_dir}")


In [36]:
coco_to_masks("train")
coco_to_masks("valid")
coco_to_masks("test")


📄 Annotation: train\_annotations.coco.json
🖼 Images: train\images
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Processing train: 100%|██████████| 167/167 [00:00<00:00, 355.92it/s]


✅ Masks saved in train\masks
📄 Annotation: valid\_annotations.coco.json
🖼 Images: valid\images
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Processing valid: 100%|██████████| 16/16 [00:00<00:00, 322.31it/s]


✅ Masks saved in valid\masks
📄 Annotation: test\_annotations.coco.json
🖼 Images: test\images
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Processing test: 100%|██████████| 17/17 [00:00<00:00, 374.37it/s]

✅ Masks saved in test\masks
